# Assignment 3 

Modal synthesis, filters, spectral analysis and FM synthesis 
------------------------------------------------------------


In this assignment we will explore some fundamental DSP concepts 
to better understand the concepts we covered in class. In addition, 
we will look into the MIDI communication protocol and file format for 
controlling synthesizers and storing performance information. 

Similarly to the first assignment I will use the term familiar programming 
language to refer to the ones that you probably have encountered during your 
studies: Python, C, C++, Java, and Javascript. As you probably can guess I will use the
term unfamiliar programming language to refer to any other programming
language such as: Haskell, OCaml, Prolog, Rust, Go, Julia, Ruby, C#, F#
R, etc. I will use the term computer music textual languages to refer to languages 
that have extensive support and primitives for sound and music manipulation such as 
Chuck, Supercollider, Csound, Nyquist, and Faust and visual programming languages 
for languages such as PureData and Max/MSP. 

Unless explicitly stated you can use any programming languages for implementing 
the questions. Using a computer music language moves the degree of difficulty down and using an unfamiliar programming language moves it up. For example if you implement question 6 in Max/MSP or Chuck it counts as basic rather than expected. If you implement question 6 in an unfamiliar programming languages it can count as advanced. In general, I am flexible so if you want to adjust things just let me know. Also if you think of a question of comparable difficulty that you would like do again let me know and most likely it should be ok. 

If you need access to devices ask me via email or through Discord for access to ECS602. 

In [27]:
import IPython.display as ipd
import numpy as np
import bokeh 
from bokeh.io import output_notebook
from bokeh.plotting import figure, output_file, show
from scipy.io import wavfile
import scipy.signal 

output_notebook()

Loading BokehJS ...

1. (Basic) Watch one of the videos listed under [Resources](resources.md). Select a short part of the video (something like 15-30 seconds) and make an explicit connection to any of the concepts we have covered in the course. Write a short paragraph describing the connection and provide the timing of the excerpt you considered. 

2. (Basic) Watch one of the interviews listed under [Resources](resources.md). Select a short part of the interview (something like 30-60 seconds) and make an explicit connection to any of the concepts we have covered in the course. Write a short paragraph describing the connection and provide the timing of the excerpt you considered. 


3. (Basic) Get two different coffee mugs and record the sound of hitting them with a pen. Plot the corresponding magnitude spectra (there is some code for plotting magnitude spectra in Python in the FM synthesis notebook) - you are welcome to use other tools. 

In [2]:
mug1sr, mug1 = wavfile.read('./audio/mug1.wav')
mug2sr, mug2 = wavfile.read('./audio/mug2.wav')

/tmp/ipykernel_20565/3549385007.py:2: WavFileWarning: Chunk (non-data) not understood, skipping it.
  mug2sr, mug2 = wavfile.read('./audio/mug2.wav')


In [3]:
def plot_mag_spectra(output, srate):
    mag_spectrum = abs(np.fft.rfft(output))
    p = figure()
    freqs = np.linspace(0, 0.5 * srate, len(mag_spectrum))
    max_freq_bin = int(srate / len(mag_spectrum) * 2000)
    p.line(freqs[0:max_freq_bin],mag_spectrum[0:max_freq_bin] * 2 * (1.0 / srate))
    show(p)

In [4]:
plot_mag_spectra(mug1, mug1sr)

In [5]:
plot_mag_spectra(mug2, mug2sr)

4. (Basic) Create two additive synthesis models of the coffee mug recordings from question 3. Each model should consist of 4 sinusoidal oscillators and associated envelopes. Listen to the resulting sound and comment on whether you can recognize which coffee mug is which from the additive synthesis approximation. You can use any language (including computer music languages) for this question. 

In [69]:
# From lecture notebook
def plot(data_list): 
    fig, ax = plt.subplots(figsize=(4,3))
    for data in data_list: 
        plt.plot(data)    

def envelope(segments,srate,duration): 
    nsamples = int(srate*duration)
    value = 0.0
    segment_index = 0 
    data = np.zeros(nsamples)
    segment_sample = 0 
    prev_target = 0.0

    for i in np.arange(nsamples): 
        if (segment_index < len(segments)): 
            target = segments[segment_index][0]
            ramp_time = segments[segment_index][1]
            delay_time = segments[segment_index][2]
            
            ramp_samples = (ramp_time / 1000.0) * srate 
            delay_samples = (delay_time / 1000.0) * srate
            
            if i < segment_sample + ramp_samples: 
                incr = (target-prev_target) / ramp_samples 
            elif i < segment_sample + ramp_samples + delay_samples: 
                incr = 0.0 
            else: 
                if ramp_samples != 0.0: 
                    incr = (target-prev_target) / ramp_samples 
                else: 
                    incr = 0.0 
                segment_sample = i 
                segment_index = segment_index+1 
                prev_target = target 
            value = value + incr 
        data[i] = value
    return data  

def sinusoid(freq=440.0, dur=1.0, srate=44100.0, amp=1.0, phase = 0.0): 
    t = np.linspace(0,dur,int(srate*dur))
    data = amp * np.sin(2*np.pi*freq *t+phase)
    return data

In [70]:
def synth_model(srate, dur, f0, s1, s2, s3, s4):
    penv1 = envelope(s1, srate, dur)
    penv2 = envelope(s2, srate, dur)
    penv3 = envelope(s3, srate, dur)
    penv4 = envelope(s4, srate, dur)
    osc1 = sinusoid(f0, dur=dur, srate=srate)
    osc2 = sinusoid(2*f0, dur=dur, srate=srate)
    osc3 = sinusoid(3*f0, dur=dur, srate=srate)
    osc4 = sinusoid(4*f0, dur=dur, srate=srate)
    return 0.25*(penv1 * osc1 + penv2 * osc2 + penv3 * osc3 + penv4 * osc4)

In [100]:
s1 = [(0.8, 30, 0), (1,30,50), (0.5, 10, 20), (0,20, 50)]
s2 = [(0.8, 30, 0), (0.35, 30, 50), (0.35, 30, 50), (0.35, 30, 50)]
s3 = [(0.9, 30, 0), (0.45, 30, 50), (0.2, 30, 40)]
s4 = [(0.95,30,100), (0.2, 30, 60), (0.4, 30, 50), (0, 20, 30)]

dur = 0.4
data = synth_model(mug1sr, dur, 200, s1, s2, s3, s4)
ipd.Audio(data,rate=mug1sr)

In [101]:
s1 = [(0.8, 30, 0), (1,30,50), (0.5, 10, 20), (0,20, 50)]
s2 = [(0.8, 30, 0), (0.35, 30, 50), (0.35, 30, 50), (0.35, 30, 50)]
s3 = [(0.9, 30, 0), (0.45, 30, 50), (0.2, 30, 40)]
s4 = [(0.95,30,100), (0.2, 30, 60), (0.4, 30, 50), (0, 20, 30)]

dur = 0.4
data = synth_model(mug2sr, dur, 500, s1, s2, s3, s4)
ipd.Audio(data,rate=mug2sr)

<span style="color:red">Yes, I can recognize which mug is which based on the high or low frequency, but I cannot recognize the mug sounds. This sounds more like a musical instrument.</span>

5. (Expected) Create two modal synthesis models of the coffee mug recordings from question 3. Each model should consist of 4 BiQuad filters with appropriate associated center frequencies and resonances excited by an impulse function. Listen to the resulting sound and comment on whether you can recognize which coffee mug is which from the modal synthesis approximation. You can use any language (including computer music languages) for this question. 

In [28]:
# Lecture code
def band_pass(audio, freq, q, srate): 
    b = np.zeros(3)
    a = np.zeros(3)
    # center frequency in radians 
    frad = 2 * np.pi * freq / srate 

    # bandpass formulas 
    alpha_ = np.sin(frad)/(2*q)
    b[0] = np.sin(frad)/2
    b[1] =  0
    b[2] = -np.sin(frad)/2
    a[0] = 1 + alpha_
    a[1] = -2 * np.cos(frad) 
    a[2] = 1 - alpha_
    
    # apply filter once
    filtered_audio = scipy.signal.lfilter(b, a, audio)
    return filtered_audio 

In [58]:
srate = mug1sr

impulse = np.zeros(srate)
impulse[int(srate/2)] = 1
data = band_pass(impulse, 800, 100, srate)
ipd.Audio(data, rate = srate)

In [66]:
srate = mug2sr

impulse = np.zeros(srate)
impulse[int(srate/2)] = 1
data = band_pass(impulse, 5000, 300, srate)
ipd.Audio(data, rate = srate)

<span style="color:red">Yes, I can recognize which mug is which based on the high or low frequency, but I cannot recognize the mug sounds. I adjust the center frequency based on the magnitude spectra from question 3, but the sound is not as rich as the original.</span>

6. (Expected) Compare the mangitude spectra of the original mug recordings, the additive synthesis approximation, and the modal synthesis approximation. Describe what you observe and comment on whether it corresponds to what you hear. 

7. (Expected) Synthesize a percussive sound using FM synthesis. The attack should be short and there should be many frequencies (i.e a high modulation index). Experiment with the amplitude and modulation index envelope shape until you have a good percussive sound. 

8. (Expected) Implement the parametric two pole filter used in the water bottle modal synthesis paper and described in this publication - create two audio examples showing how it can be used: https://ccrma.stanford.edu/~jos/smac03maxjos/smac03maxjos.pdf

9. (Advanced) Use the parametric two-pole filter mentioned in question 8 to create another modal synthesis approximation for the two coffee mugs of questions 5,6. Repeat the comparison between the different models both in terms of magnitude spectrums and in terms of listening. The water bottle modal synthesis paper (https://dafx2020.mdw.ac.at/proceedings/papers/DAFx2020_paper_24.pdf) describes a process for estimating the mode decay that you can follow for the coffee mug recordings. 

10. (Advanced) Implemented a real-time version of any of the synthesis models you developed in this assignments. You can use any language/programming environment for this question. 

